# 6.2 O processo de Wiener e o movimento browniano

In [12]:
import numpy as np
import pandas as pd
from scipy import stats as st
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import random
import warnings

warnings.filterwarnings("ignore")

In [63]:
np.random.normal()

0.3201457581057413

In [55]:
# A nossa classe para o movimento browniano

class mov_brow():

    def __init__(self, x = 0):        
        self.x = float(x)
    
    def normal_dist(self, n_step = 100):
        w = np.ones(n_step)*self.x
        for i in range(1, n_step):
            y = np.random.normal()
            w[i] = w[i-1]+(y/np.sqrt(n_step)) # Processo de Weiner
        return w
    
    def preco(self, s0 = 10, mu = 0.2
              , sigma = 0.25, deltaT = 252, dt = 1):
        
        # Processo de criação do preço:
        # S(t) = S(0).exp{(mu-(sigma^2/2).t)+sigma.W(t)}
        # s0: Preço inicial do ativo
        # mu: 'Drift' do ativo (para cima ou para baixo)
        # sigma: Conhecido como volatilidade
        # deltaT: Período de tempo para o cálculo em semanas
        # dt: granularidade do período
        
        n_step = int(deltaT/dt)
        tempo = np.linspace(0, deltaT, num = n_step)
        preços = (mu-(sigma**2/2))*tempo
        weiner = sigma*self.normal_dist(n_step)
        s = s0*(np.exp(preços + weiner))
        return tempo, s

In [56]:
# Inicializa a simulação
sim = mov_brow()

In [64]:
# Roda a simulação e plota o gráfico
media = 0.02
sigma = 0.2

tempo, ativo1 = sim.preco(mu = media, sigma = sigma)
tempo, ativo2 = sim.preco(mu = media, sigma = sigma)
tempo, ativo3 = sim.preco(mu = media, sigma = sigma)

fig = make_subplots(rows = 1, cols = 1
                    , shared_xaxes = True
                    , vertical_spacing = 0.05)

fig.add_trace(go.Scatter(x = tempo, y = ativo1
                                , name = "Ativo 1"
                                , line = dict(color = "blue"))
              , row = 1, col = 1)

fig.add_trace(go.Scatter(x = tempo, y = ativo2
                                , name = "Ativo 2"
                                , line = dict(color = "green"))
              , row = 1, col = 1)

fig.add_trace(go.Scatter(x = tempo, y = ativo3
                                , name = "Ativo 3"
                                , line = dict(color = "orange"))
              , row = 1, col = 1)


fig.update_layout(height = 600, width = 800
                  , title_text = "Movimento Browniano de 3 ativos de estatísticas iguais"
                  , font_color = "blue"
                  , title_font_color = "black"
                  , xaxis_title = "Tempo"
                  , yaxis_title = "Preço"
                  , showlegend = True
                  , legend_title = "Simulações"
                  , font = dict(size = 15, color = "Black")
                 )

fig.show()

In [88]:
# Roda a simulação e plota o gráfico para estatísticas diferentes


tempo, ativo1 = sim.preco(mu = 0.02, sigma = 0.2)
tempo, ativo2 = sim.preco(mu = 0.021, sigma = 0.2)
tempo, ativo3 = sim.preco(mu = 0.019, sigma = 0.2)

fig = make_subplots(rows = 1, cols = 1
                    , shared_xaxes = True
                    , vertical_spacing = 0.05)

fig.add_trace(go.Scatter(x = tempo, y = ativo1
                                , name = "Ativo 1"
                                , line = dict(color = "blue"))
              , row = 1, col = 1)

fig.add_trace(go.Scatter(x = tempo, y = ativo2
                                , name = "Ativo 2"
                                , line = dict(color = "green"))
              , row = 1, col = 1)

fig.add_trace(go.Scatter(x = tempo, y = ativo3
                                , name = "Ativo 3"
                                , line = dict(color = "orange"))
              , row = 1, col = 1)


fig.update_layout(height = 600, width = 800
                  , title_text = "Movimento Browniano de 3 ativos de estatísticas iguais"
                  , font_color = "blue"
                  , title_font_color = "black"
                  , xaxis_title = "Tempo"
                  , yaxis_title = "Preço"
                  , showlegend = True
                  , legend_title = "Simulações"
                  , font = dict(size = 15, color = "Black")
                 )

fig.show()

In [89]:
# Capturando os dados para a análise

def busca_titulos_tesouro_direto():
    url = 'https://www.tesourotransparente.gov.br/ckan/dataset/df56aa42-484a-4a59-8184-7676580c81e3/resource/796d2059-14e9-44e3-80c9-2d9e30b405c1/download/PrecoTaxaTesouroDireto.csv'
    df  = pd.read_csv(url, sep=';', decimal=',')
    df['Data Vencimento'] = pd.to_datetime(df['Data Vencimento'], dayfirst=True)
    df['Data Base']       = pd.to_datetime(df['Data Base'], dayfirst=True)
    multi_indice = pd.MultiIndex.from_frame(df.iloc[:, :3])
    df = df.set_index(multi_indice).iloc[: , 3:]  
    return df

In [8]:
titulos = busca_titulos_tesouro_direto()
ipca = titulos.loc[('Tesouro IPCA+', '2026-08-15')]
ipca.sort_index(inplace=True)

In [90]:
ipca["Retornos"] = ipca["PU Base Manha"].pct_change(1)

In [91]:
ipca["2022"]["PU Base Manha"][-1]

3197.5

In [94]:
# Roda a simulação com dados de um título

tempo, titulo1 = sim.preco(s0 = ipca["2022"]["PU Base Manha"][-1]
                           , mu = ipca["2022"]["Retornos"].mean()
                           , sigma = ipca["2022"]["Retornos"].std())

tempo, titulo2 = sim.preco(s0 = ipca["2022"]["PU Base Manha"][-1]
                           , mu = ipca["2022"]["Retornos"].mean()
                           , sigma = ipca["2022"]["Retornos"].std())

tempo, titulo3 = sim.preco(s0 = ipca["2022"]["PU Base Manha"][-1]
                           , mu = ipca["2022"]["Retornos"].mean()
                           , sigma = ipca["2022"]["Retornos"].std())

fig = make_subplots(rows = 1, cols = 1
                    , shared_xaxes = True
                    , vertical_spacing = 0.05)

fig.add_trace(go.Scatter(x = tempo, y = titulo1
                                , name = "Ipca 2026 - Sim1"
                                , line = dict(color = "blue"))
              , row = 1, col = 1)

fig.add_trace(go.Scatter(x = tempo, y = titulo2
                                , name = "Ipca 2026 - Sim2"
                                , line = dict(color = "red"))
              , row = 1, col = 1)

fig.add_trace(go.Scatter(x = tempo, y = titulo3
                                , name = "Ipca 2026 - Sim3"
                                , line = dict(color = "green"))
              , row = 1, col = 1)


fig.update_layout(height = 600, width = 800
                  , title_text = "Movimento Browniano aplicado ao título Tesouro IPCA 2026"
                  , font_color = "blue"
                  , title_font_color = "black"
                  , xaxis_title = "Tempo"
                  , yaxis_title = "Preço"
                  , showlegend = True
                  , legend_title = "Simulações"
                  , font = dict(size = 15, color = "Black")
                 )

fig.show()